In [23]:
import os
import time
import requests
import pandas as pd

api_key = os.environ.get('riot_api_key')
base_url = "https://na1.api.riotgames.com"
solo_duo = "RANKED_SOLO_5x5"

print(api_key)
print(base_url)

RGAPI-f39807c0-1dc0-4741-a2b4-9953668d5da7
https://na1.api.riotgames.com


In [24]:
def getChallengers() -> pd.DataFrame:
	url = f"{base_url}/lol/league/v4/challengerleagues/by-queue/{solo_duo}?api_key={api_key}"

	response = requests.get(url)

	if response.status_code == 200:
			data = response.json()
			players = data['entries']

			return pd.DataFrame(players)


In [25]:
def getMatches(puuid):
	url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&start=0&count=20&api_key={api_key}"

	response = requests.get(url)

	if response.status_code == 200:
			data = response.json()

			return data
	else:
		print(response)

In [26]:
blueTeamId = 100
redTeamId = 200
def getWinner(teams):
	for t in teams:
		if t['teamId'] == blueTeamId:
			return 0 if t['win'] else 1

def getChampionID(participants, side, position):
	for p in participants:
		if p['teamId'] == side and p['teamPosition'] == position:
			return p['championId']


def getMatchData(matchId):
	url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}"

	response = requests.get(url)

	if response.status_code == 200:
		match_data = response.json()

		scrubbed_match_data = {
			'matchId': match_data['metadata']['matchId'],
			'gameDurationSeconds': match_data['info']['gameDuration'],
			'championBlueTop': getChampionID(match_data['info']['participants'], blueTeamId, 'TOP'),
			'championBlueJG': getChampionID(match_data['info']['participants'], blueTeamId, 'JUNGLE'),
			'championBlueMid': getChampionID(match_data['info']['participants'], blueTeamId, 'MIDDLE'),
			'championBlueBot': getChampionID(match_data['info']['participants'], blueTeamId, 'BOTTOM'),
			'championBlueSup': getChampionID(match_data['info']['participants'], blueTeamId, 'UTILITY'),
			'championRedTop': getChampionID(match_data['info']['participants'], redTeamId, 'TOP'),
			'championRedJG': getChampionID(match_data['info']['participants'], redTeamId, 'JUNGLE'),
			'championRedMid': getChampionID(match_data['info']['participants'], redTeamId, 'MIDDLE'),
			'championRedBot': getChampionID(match_data['info']['participants'], redTeamId, 'BOTTOM'),
			'championRedSup': getChampionID(match_data['info']['participants'], redTeamId, 'UTILITY'),
			'winner': getWinner(match_data['info']['teams']) # 0 = Blue side, 1 = Red Side
		}

		return scrubbed_match_data

In [27]:
# Remove Duplicates
match_ids = pd.read_csv("MatchIDs.csv")
match_ids = match_ids.drop("Unnamed: 0", axis=1)

match_ids = match_ids.drop_duplicates()


In [ ]:
match_data = getMatchData(match_ids.iloc[0].match_ids)

print(match_data)

match_datas = []

for index, row in match_ids.iterrows():
	print(row.match_ids)

	match_data = getMatchData(row.match_ids)

	if (index + 1) % 20 == 0:
		print(len(match_ids))
		time.sleep(30) # Sleep 30 seconds so that it won't hit the rate limit




{'matchId': 'NA1_5242178677', 'gameDurationSeconds': 1437, 'championBlueTop': 6, 'championBlueJG': 64, 'championBlueMid': 126, 'championBlueBot': 110, 'championBlueSup': 888, 'championRedTop': 266, 'championRedJG': 254, 'championRedMid': 31, 'championRedBot': 222, 'championRedSup': 201, 'winner': 1}
0
match_ids    NA1_5242178677
Name: 0, dtype: object
1
match_ids    NA1_5241910659
Name: 1, dtype: object
2
match_ids    NA1_5241896512
Name: 2, dtype: object
3
match_ids    NA1_5241251868
Name: 3, dtype: object
4
match_ids    NA1_5241210985
Name: 4, dtype: object
5
match_ids    NA1_5241099123
Name: 5, dtype: object
6
match_ids    NA1_5241059102
Name: 6, dtype: object
7
match_ids    NA1_5241039930
Name: 7, dtype: object
8
match_ids    NA1_5240996452
Name: 8, dtype: object
9
match_ids    NA1_5240978553
Name: 9, dtype: object
10
match_ids    NA1_5240961648
Name: 10, dtype: object
11
match_ids    NA1_5240938796
Name: 11, dtype: object
12
match_ids    NA1_5240780967
Name: 12, dtype: object
13
m

In [29]:
# challengers = getChallengers()

# match_ids = []

# print(len(challengers))

# for index, row in challengers.iterrows():
# 	matches = getMatches(row.puuid)
# 	match_ids.extend(matches)

# 	if (index + 1) % 20 == 0:
# 		print(len(match_ids))
# 		time.sleep(30) # Sleep 30 seconds so that it won't hit the rate limit

# print(len(match_ids)) # should be 6000

# df = pd.DataFrame({ 'match_ids': match_ids })

# print(df.shape)

# df.to_csv("./MatchIDs.csv")
